<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия №3
<center>Автор материала: программист-исследователь Mail.Ru Group Юрий Кашницкий

# <center> Домашнее задание № 8
## <center> Vowpal Wabbit в задаче классификации тегов вопросов на Stackoverflow

## План
    1. Введение
    2. Описание данных
    3. Предобработка данных
    4. Обучение и проверка моделей
    5. Заключение

### 1. Введение

В этом задании вы будете делать примерно то же, что я каждую неделю –  в Mail.Ru Group: обучать модели на выборке в несколько гигабайт. Задание можно выполнить и на Windows с Python, но я рекомендую поработать под \*NIX-системой (например, через Docker) и активно использовать язык bash.
Немного снобизма (простите, но правда): если вы захотите работать в лучших компаниях мира в области ML, вам все равно понадобится опыт работы с bash под UNIX.

[Веб-форма](https://docs.google.com/forms/d/1VaxYXnmbpeP185qPk2_V_BzbeduVUVyTdLPQwSCxDGA/edit) для ответов.

Для выполнения задания понадобится установленный Vowpal Wabbit (уже есть в докер-контейнере курса, см. инструкцию в Wiki [репозитория](https://github.com/Yorko/mlcourse_open) нашего курса) и примерно 70 Гб дискового пространства. Я тестировал решение не на каком-то суперкомпе, а на Macbook Pro 2015 (8 ядер, 16 Гб памяти), и самая тяжеловесная модель обучалась около 12 минут, так что задание реально выполнить и с простым железом. Но если вы планируете когда-либо арендовать сервера Amazon, можно попробовать это сделать уже сейчас.

Материалы в помощь:
 - интерактивный [тьюториал](https://www.codecademy.com/en/courses/learn-the-command-line/lessons/environment/exercises/bash-profile) CodeAcademy по утилитам командной строки UNIX (примерно на час-полтора)
 - [статья](https://habrahabr.ru/post/280562/) про то, как арендовать на Amazon машину (еще раз: это не обязательно для выполнения задания, но будет хорошим опытом, если вы это делаете впервые)

### 2. Описание данных

Имеются 10 Гб вопросов со StackOverflow – [скачайте](https://drive.google.com/file/d/1ZU4J3KhJDrHVMj48fROFcTsTZKorPGlG/view) и распакуйте архив. 

Формат данных простой:<br>
<center>*текст вопроса* (слова через пробел) TAB *теги вопроса* (через пробел)

Здесь TAB – это символ табуляции.
Пример первой записи в выборке:

In [2]:
!head -1 ../../data_external/stackoverflow.10kk.tsv

 is there a way to apply a background color through css at the tr level i can apply it at the td level like this my td background color e8e8e8 background e8e8e8 however the background color doesn t seem to get applied when i attempt to apply the background color at the tr level like this my tr background color e8e8e8 background e8e8e8 is there a css trick to making this work or does css not natively support this for some reason 	css css3 css-selectors


Здесь у нас текст вопроса, затем табуляция и теги вопроса: *css, css3* и *css-selectors*. Всего в выборке таких вопросов 10 миллионов. 

In [4]:
%%time
!wc -l ../../data_external/stackoverflow.10kk.tsv

10000000 ../../data_external/stackoverflow.10kk.tsv
CPU times: user 52 ms, sys: 8 ms, total: 60 ms
Wall time: 2.07 s


Обратите внимание на то, что такие данные я уже не хочу загружать в оперативную память и, пока можно, буду пользоваться эффективными утилитами UNIX –  head, tail, wc, cat, cut и прочими.

In [6]:
!sed -i 1,7d ../../data_external/stackoverflow.10kk.tsv

In [7]:
!head -1 ../../data_external/stackoverflow.10kk.tsv

 i need to write a web application which works only with mysql database but all actual data lies within an oracle so now i m looking for some way to syncronize partially or use some automated tools to do that or just for a best practice for that case to be clear there s no way to use oracle directly from web applications due to security policies any advices 	mysql oracle


### 3. Предобработка данных

Давайте выберем в наших данных все вопросы с тегами *javascript, java, python, ruby, php, c++, c#, go, scala* и  *swift* и подготовим обучающую выборку в формате Vowpal Wabbit. Будем решать задачу 10-классовой классификации вопросов по перечисленным тегам.

Вообще, как мы видим, у каждого вопроса может быть несколько тегов, но мы упростим себе задачу и будем у каждого вопроса выбирать один из перечисленных тегов либо игнорировать вопрос, если таковых тегов нет. 
Но вообще VW поддерживает multilabel classification (аргумент  --multilabel_oaa).
<br>
<br>
Реализуйте в виде отдельного файла `preprocess.py` код для подготовки данных. Он должен отобрать строки, в которых есть перечисленные теги, и переписать их в отдельный файл в формат Vowpal Wabbit. Детали:
 - скрипт должен работать с аргументами командной строки: с путями к файлам на входе и на выходе
 - строки обрабатываются по одной (можно использовать tqdm для подсчета числа итераций)
 - если табуляций в строке нет или их больше одной, считаем строку поврежденной и пропускаем
 - в противном случае смотрим, сколько в строке тегов из списка *javascript, java, python, ruby, php, c++, c#, go, scala* и  *swift*. Если ровно один, то записываем строку в выходной файл в формате VW: `label | text`, где `label` – число от 1 до 10 (1 - *javascript*, ... 10 – *swift*). Пропускаем те строки, где интересующих тегов больше или меньше одного 
 - из текста вопроса надо выкинуть двоеточия и вертикальные палки, если они есть – в VW это спецсимволы

In [1]:
import os
from tqdm import tqdm
from time import time
import numpy as np
from sklearn.metrics import accuracy_score

In [2]:
tags = ['javascript', 'java', 'python', 'ruby', 'php', 'c++', 'c#', 'go', 'scala', 'swift']

In [3]:
fin = "../../data_external/stackoverflow.10kk.tsv"
reader = open(fin, 'r')

In [17]:
line = reader.readline()
text_labels = line[:-1].split("\t")
if len(text_labels) != 2:
#     n_corrupted_lines += 1
    raise Exception("Corrupted")
text, labels = text_labels
labels = labels.split(' ')
found_tags = [l for l in labels if l in tags]
if len(found_tags) != 1:
    raise Exception("No tags : {}".format(found_tags))
text = text.strip().replace(':', ' ').replace('|',' ')
label = tags.index(found_tags[0])
print("{} | {}\n".format(label, text))
# n_written_lines += 1

1 | i am new to this forum and also to ganglia we are trying to capture jvm metrics through ganglia i have followed the steps through this link https github com ganglia jmxetric high level installation unzip the archive add the following to your jvm java javaagent lt path gt jmxetric jar host port config process usual java main class demo quickstart this example works for version 1 0 6 of jmxetric and gmetric4j ensure you have a gmond running on localhost 8649 pgrep gmond should return a valid pid nc localhost 8649 dumps some xml to stdout git clone same link as above download jmxetric jar gmetric4j jar and oncrpc 1 0 7 jar all into the same directory cd jmxetric in bash do export config host localhost port 8649 wireformat31x true config etc jmxetric xml java djava util logging config file etc logging properties cp gmetric4j 1 0 6 jar oncrpc 1 0 7 jar jmxetric 1 0 6 jar javaagent jmxetric 1 0 4 jar config info ganglia jmxetric jmxetricagent but when i add the jvm argument in was its no

In [123]:
text = text.strip().replace(':', ' ').replace('|',' ')
label = tags.index(found_tags[0])
"{} | {}".format(label, text)

'1 | i have a collection of objects that are guaranteed to be distinct in particular indexed by a unique integer id i also know exactly how many of them there are and the number won t change and was wondering whether array would have a notable performance advantage over hashset for storing retrieving said elements on paper array guarantees constant time insertion since i know the size ahead of time and retrieval but the code for hashset looks much cleaner and adds some flexibility so i m wondering if i m losing anything performance wise using it at least theoretically'

In [18]:
reader.close()

In [70]:
%%writefile preprocess.py
import os
import sys
from tqdm import tqdm
from time import time
import numpy as np
from sklearn.metrics import accuracy_score

tags = ['javascript', 'java', 'python', 'ruby', 'php', 'c++', 'c#', 'go', 'scala', 'swift']

def main(fin, fout):
    
    n_corrupted_lines = 0
    n_written_lines = 0
    
    with tqdm(total=9999994) as pbar:
        with open(fin, 'r') as reader:
            with open(fout, 'bw') as writer:
                while True:
                    line = reader.readline()
                    pbar.update(1)                    
                    if len(line) == 0:
                        break
                    text_labels = line[:-1].split("\t")
                    if len(text_labels) != 2:
                        n_corrupted_lines += 1
                        continue
                    text, labels = text_labels
                    labels = labels.split(' ')
                    found_tags = [l for l in labels if l in tags]
                    if len(found_tags) != 1:
                        continue
                    text = text.strip().replace(':', ' ').replace('|',' ')
                    label = tags.index(found_tags[0]) + 1
                    writer.write("{} | {}\n".format(label, text).encode('utf-8'))
                    n_written_lines += 1
    print("{} lines selected, {} lines corrupted.".format(n_written_lines, n_corrupted_lines))


if __name__ == '__main__':
    fin = sys.argv[1]
    fout = sys.argv[2]
    assert os.path.exists(fin)
    if fin is None or fout is None:
        print('Incorrect args:\nfin - {}\nfout - {}'.format(fin, fout))
        sys.exit(-1)
    main(fin, fout)
    print('Done')

Overwriting preprocess.py


In [71]:
!python -m py_compile preprocess.py

Должно получиться вот такое число строк – 4389054. 10 Гб у меня обработались примерно за 2 минуты.

In [72]:
!python3 preprocess.py ../../data_external/stackoverflow.10kk.tsv ../../data_external/stackoverflow.vw

100%|#############################| 9999994/9999994 [00:58<00:00, 170925.14it/s]
4389056 lines selected, 0 lines corrupted.
Done


In [73]:
%%time
!wc -l ../../data_external/stackoverflow.vw

4389056 ../../data_external/stackoverflow.vw
CPU times: user 28 ms, sys: 8 ms, total: 36 ms
Wall time: 1.18 s


In [6]:
# !gzip ../../data_external/stackoverflow.10kk.tsv

Поделите выборку на обучающую, проверочную и тестовую части в равной пропорции - по  1463018 в каждый файл. Перемешивать не надо, первые 1463018 строк должны пойти в обучающую часть `stackoverflow_train.vw`, последние 1463018 – в тестовую `stackoverflow_test.vw`, оставшиеся – в проверочную `stackoverflow_valid.vw`. 

Также сохраните векторы ответов для проверочной и тестовой выборки в отдельные файлы `stackoverflow_valid_labels.txt` и `stackoverflow_test_labels.txt`.

Тут вам помогут утилиты `head`, `tail`, `split`, `cat` и `cut`.

In [29]:
!split --help

Usage: split [OPTION]... [FILE [PREFIX]]
Output pieces of FILE to PREFIXaa, PREFIXab, ...;
default size is 1000 lines, and default PREFIX is 'x'.

With no FILE, or when FILE is -, read standard input.

Mandatory arguments to long options are mandatory for short options too.
  -a, --suffix-length=N   generate suffixes of length N (default 2)
      --additional-suffix=SUFFIX  append an additional SUFFIX to file names
  -b, --bytes=SIZE        put SIZE bytes per output file
  -C, --line-bytes=SIZE   put at most SIZE bytes of records per output file
  -d                      use numeric suffixes starting at 0, not alphabetic
      --numeric-suffixes[=FROM]  same as -d, but allow setting the start value
  -e, --elide-empty-files  do not generate empty output files with '-n'
      --filter=COMMAND    write to shell COMMAND; file name is $FILE
  -l, --lines=NUMBER      put NUMBER lines/records per output file
  -n, --number=CHUNKS     generate CHUNKS output files; see explanation below
  -t, 

In [74]:
!split -l 1463018 --verbose --additional-suffix=.vw ../../data_external/stackoverflow.vw ../../data_external/stackoverflow_


creating file '../../data_external/stackoverflow_aa.vw'
creating file '../../data_external/stackoverflow_ab.vw'
creating file '../../data_external/stackoverflow_ac.vw'
creating file '../../data_external/stackoverflow_ad.vw'


In [75]:
!mv ../../data_external/stackoverflow_aa.vw ../../data_external/stackoverflow_train.vw
!mv ../../data_external/stackoverflow_ab.vw ../../data_external/stackoverflow_valid.vw
!mv ../../data_external/stackoverflow_ac.vw ../../data_external/stackoverflow_test.vw
!cat ../../data_external/stackoverflow_ad.vw >> ../../data_external/stackoverflow_test.vw
!rm ../../data_external/stackoverflow_ad.vw

In [76]:
!wc -l ../../data_external/stackoverflow_*.vw

   1463020 ../../data_external/stackoverflow_test.vw
   1463018 ../../data_external/stackoverflow_train.vw
   1463018 ../../data_external/stackoverflow_valid.vw
   4389056 total


In [47]:
# !du -h -a -d 1 ../../data_external/

In [50]:
!cut --help

Usage: cut OPTION... [FILE]...
Print selected parts of lines from each FILE to standard output.

With no FILE, or when FILE is -, read standard input.

Mandatory arguments to long options are mandatory for short options too.
  -b, --bytes=LIST        select only these bytes
  -c, --characters=LIST   select only these characters
  -d, --delimiter=DELIM   use DELIM instead of TAB for field delimiter
  -f, --fields=LIST       select only these fields;  also print any line
                            that contains no delimiter character, unless
                            the -s option is specified
  -n                      (ignored)
      --complement        complement the set of selected bytes, characters
                            or fields
  -s, --only-delimited    do not print lines not containing delimiters
      --output-delimiter=STRING  use STRING as the output delimiter
                            the default is to use the input delimiter
  -z, --zero-terminated    line delimite

In [77]:
!cat ../../data_external/stackoverflow_valid.vw | cut -d\| -f 1 > ../../data_external/stackoverflow_valid_labels.txt

In [78]:
!cat ../../data_external/stackoverflow_test.vw | cut -d\| -f 1 > ../../data_external/stackoverflow_test_labels.txt

In [79]:
!head ../../data_external/stackoverflow_valid_labels.txt

5 
5 
6 
1 
7 
7 
7 
4 
5 
3 


In [80]:
!head ../../data_external/stackoverflow_test_labels.txt

7 
2 
9 
1 
7 
5 
7 
9 
7 
2 


### 4. Обучение и проверка моделей

Обучите Vowpal Wabbit на выборке `stackoverflow_train.vw` 9 раз, перебирая параметры passes (1,3,5), ngram (1,2,3).
Остальные параметры укажите следующие: bit_precision=28 и seed=17. Также скажите VW, что это 10-классовая задача.

Проверяйте долю правильных ответов на выборке `stackoverflow_valid.vw`. Выберите лучшую модель и проверьте качество на выборке `stackoverflow_test.vw`.

In [1]:
params = [
    "--passes 1 --ngram 1",
    "--passes 1 --ngram 2",
    "--passes 1 --ngram 3",
    
    "--passes 3 --ngram 1",
    "--passes 3 --ngram 2",
    "--passes 3 --ngram 3",
    
    "--passes 5 --ngram 1",
    "--passes 5 --ngram 2",
    "--passes 5 --ngram 3"
]

In [12]:
!ls

hw1_session3_data_analysis_pandas.ipynb
hw2_session3_visual_data_analysis.ipynb
hw3_session3_decision_trees.ipynb
hw3_session3_decision_trees_solution.ipynb
hw3_session3_optional_implement_dt.ipynb
hw4_session3_stackoverflow_logistic_regression.ipynb
hw5_session3_rf_logit_scoring_texts.ipynb
hw6
hw6_session3_part1_alice_beat_baseline.ipynb
hw6_session3_part1_alice_beat_baseline_RF.ipynb
hw6_session3_part1_alice_beat_baseline_basics.ipynb
hw6_session3_part1_alice_beat_baseline_tsfresh.ipynb
hw6_session3_part2_medium_beat_baseline.ipynb
hw7_session3_unsupervised_learning.ipynb
hw8_session3_vw_stackoverflow_tags_10mln.ipynb
preprocess.py
preprocess.pyc


In [13]:
import os
import numpy as np
from sklearn.metrics import accuracy_score


for i, p in enumerate(params):
    print("\n\n")
    print(p)
    
    !vw --oaa 10 -d ../../data_external/stackoverflow_train.vw -f vw_model_{i}.vw -b 28 --random_seed 17 -c {p}
    !vw -t -i vw_model_{i}.vw -d ../../data_external/stackoverflow_valid.vw -p vw_model_{i}_pred.csv --random_seed 17 --quiet
    
    vw_pred = np.loadtxt('vw_model_{}_pred.csv'.format(i))
    test_labels = np.loadtxt('../../data_external/stackoverflow_valid_labels.txt')
    print("Accuracy: ", accuracy_score(test_labels, vw_pred))




--passes 1 --ngram 1
Generating 1-grams for all namespaces.
final_regressor = vw_model_0.vw
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
using cache_file = ../../data_external/stackoverflow_train.vw.cache
ignoring text input in favor of cache input
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        1        1       59
0.500000 1.000000            2            2.0        7        1       88
0.750000 1.000000            4            4.0        1        7      143
0.875000 1.000000            8            8.0        5        7      446
0.875000 0.875000           16           16.0        6        7      331
0.812500 0.750000           32           32.0        2        7      236
0.765625 0.718750           64           64.0        1        1      288
0.687500 0.609375          128          128.0      

0.162598 0.134094        65536        65536.0        7        3       61
0.141228 0.119858       131072       131072.0        7        7       80
0.124710 0.108192       262144       262144.0        5        5      158
0.112028 0.099346       524288       524288.0        7        7      149
0.103410 0.094791      1048576      1048576.0        3        3      152
0.095728 0.095728      2097152      2097152.0        7        7       42 h

finished run
number of examples per pass = 1316717
passes used = 3
weighted example sum = 3950151.000000
weighted label sum = 0.000000
average loss = 0.087621 h
total feature number = 787979784
Accuracy:  0.9142737820040492



--passes 3 --ngram 2
Generating 2-grams for all namespaces.
final_regressor = vw_model_4.vw
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
using cache_file = ../../data_external/stackoverflow_train.vw.cache
ignoring text input in favor of cache input
num sources = 1
average  since     

0.750000 1.000000            4            4.0        1        7      284
0.875000 1.000000            8            8.0        5        7      890
0.750000 0.625000           16           16.0        2        2      156
0.750000 0.750000           32           32.0        5        1      118
0.703125 0.656250           64           64.0        2        5      272
0.640625 0.578125          128          128.0        7        6      358
0.574219 0.507812          256          256.0        1        7      196
0.523438 0.472656          512          512.0        6        6      310
0.443359 0.363281         1024         1024.0        3        3      268
0.365723 0.288086         2048         2048.0        1        1       80
0.296875 0.228027         4096         4096.0        3        3      120
0.250000 0.203125         8192         8192.0        2        2      216
0.209595 0.169189        16384        16384.0        4        6      102
0.175323 0.141052        32768        32768.0      

<font color='red'> Вопрос 1.</font> Какое сочетание параметров дает наибольшую долю правильных ответов на проверочной выборке `stackoverflow_valid.vw`?
- Биграммы и 3 прохода по выборке
- Триграммы и 5 проходов по выборке
- Биграммы и 1 проход по выборке
- Униграммы и 1 проход по выборке

In [ ]:
# Ответ: 3
# Биграммы и 1 проход по выборке

Проверьте лучшую (по доле правильных ответов на валидации) модель на тестовой выборке. 

In [14]:
i = 1

!vw -t -i vw_model_{i}.vw -d ../../data_external/stackoverflow_test.vw -p vw_model_{i}_test_pred.csv --random_seed 17 --quiet

vw_pred = np.loadtxt('vw_model_{}_test_pred.csv'.format(i))
test_labels = np.loadtxt('../../data_external/stackoverflow_test_labels.txt')
print("Accuracy: ", accuracy_score(test_labels, vw_pred))

Accuracy:  0.9311321786441744


<font color='red'> Вопрос 2.</font> Как соотносятся доли правильных ответов лучшей (по доле правильных ответов на валидации) модели на проверочной и на тестовой выборках? (здесь % – это процентный пункт, т.е., скажем, снижение с 50% до 40% – это на 10%, а не 20%).
- На тестовой ниже примерно на 2%
- На тестовой ниже примерно на 3%
- Результаты почти одинаковы – отличаются меньше чем на 0.5%

In [16]:
(0.9311321786441744 - 0.9309892291140642) * 100

0.0001429495301101813

In [ ]:
# Ответ : 3

Обучите VW с параметрами, подобранными на проверочной выборке, теперь на объединении обучающей и проверочной выборок. Посчитайте долю правильных ответов на тестовой выборке. 

In [17]:
!cp ../../data_external/stackoverflow_train.vw ../../data_external/stackoverflow_trainval.vw
!cat ../../data_external/stackoverflow_valid.vw >> ../../data_external/stackoverflow_trainval.vw

In [18]:
!wc -l ../../data_external/stackoverflow_trainval.vw

2926036 ../../data_external/stackoverflow_trainval.vw


In [19]:
p = "--passes=1 --ngram=2"
!vw --oaa 10 -d ../../data_external/stackoverflow_trainval.vw -f vw_model_passes=1_ngram=2_trainval.vw -b 28 --random_seed 17 -c {p} --quiet
!vw -t -i vw_model_passes=1_ngram=2_trainval.vw -d ../../data_external/stackoverflow_test.vw -p vw_model_passes=1_ngram=2_test_pred.csv --random_seed 17 --quiet

vw_pred = np.loadtxt('vw_model_passes=1_ngram=2_test_pred.csv')
test_labels = np.loadtxt('../../data_external/stackoverflow_test_labels.txt')
print("Accuracy: ", accuracy_score(test_labels, vw_pred))

Accuracy:  0.9352701945291246


<font color='red'> Вопрос 3.</font> На сколько процентных пунктов повысилась доля правильных ответов модели после обучения на вдвое большей выборке (обучающая `stackoverflow_train.vw` + проверочная `stackoverflow_valid.vw`) по сравнению с моделью, обученной только на `stackoverflow_train.vw`?
 - 0.1%
 - 0.4%
 - 0.8%
 - 1.2%

In [20]:
(0.9352701945291246 - 0.9309892291140642)*100

0.42809654150604004

In [21]:
# Ответ: 2

### 5. Заключение

В этом задании мы только познакомились с Vowpal Wabbit. Что еще можно попробовать:
 - Классификация с несколькими ответами (multilabel classification, аргумент  `multilabel_oaa`) – формат данных тут как раз под такую задачу
 - Настройка параметров VW с hyperopt, авторы библиотеки утверждают, что качество должно сильно зависеть от параметров изменения шага градиентного спуска (`initial_t` и `power_t`). Также можно потестировать разные функции потерь – обучать логистическую регресиию или линейный SVM
 - Познакомиться с факторизационными машинами и их реализацией в VW (аргумент `lrq`)